# Import

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os
from typing import Tuple


# Forward Pass

## Weights and bias initialization

In [4]:
def WeightBaisInit(input_dim: int,output_dim: int) -> Tuple[np.ndarray,np.ndarray]:
    
    '''
    Initialization of weights and bias can be changed further if we need more functionalities
    '''
    
    w_init = np.random.rand(input_dim,output_dim)
    b_init = np.random.rand(output_dim,)

    return w_init,b_init

## Linear Layer Forward

In [ ]:
def linear_layer_forward(x: np.ndarray,input_dim: int,output_dim: int):

    '''
    Computes forward pass for simple linear layer 
    

    Input: 
    x: Numpy array containing input data, N x H x W we flatten last two dimensions so its N x D
    input_dim: Input dimensions of hidden layer
    output_dim: Output dimensions of hidden layer
    
    Output:


    output: Numpy array after matrix multiplication, N x M
    
    '''
    
    assert(input_dim==x.shape[1])
    #Initialize weights and bias for this layer

    w,b = WeightBaisInit(input_dim,output_dim)

    output =  (x @ w) + b
    

    cache = (x,w,b)
    return output, cache

## Sigmoid Function Forward

In [ ]:
def sigmoid_activation_forward(x: np.ndarray) -> (np.ndarray):

    '''
    
    Apply sigmoid function on given input
    
    Input:

    x: Numpy array, NxD

    Output:

    output: Numpy array after sigmoid activation , NxD

    '''

    #Check if we need to normalize the input before passing to sigmoid

    output  = 1 / (1 + np.exp(-x))

    cache = x

    return output,cache

## Softmax Layer Forward

In [ ]:
def softmax_activation_forward(x: np.ndarray) -> (np.ndarray):
    '''
    
    Apply softmax function on given input
    
    Input:

    x: Numpy array, NxD

    Output:

    output: Numpy array after softmax activation , NxD

    '''

    #Check if we need to normalize the input before passing to softmax

    normalize_input = x - np.max(x)

    output = np.exp(normalize_input)/(np.sum(np.exp(normalize_input)))

    cache = normalize_input

    return output,cache

# Loss

In [5]:
def MSE_loss(y_pred: np.ndarray, y: np.ndarray) -> float:
    
    '''
    Computes Mean squared error loss
    
    Input:

    y: Numpy array containg ground truth labels, NxD
    y_pred: Numpy array from network, NxD

    Output:

    loss: scaler, mean squared error loss
    gradient: gradient w.r.t y_pred used for backward pass

    '''

    N = y.shape[0]
    loss = (np.sum(((y- y_pred) ** 2))) / N


    gradient = -(2/N) * (np.sum(y-y_pred))

    return loss,gradient

# Backward Pass

# Model

# Training

# Testing/Validation

# Plots